In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [4]:
import pymysql

In [5]:
conn=pymysql.connect(host='127.0.0.1',user='root',password='634155')
cursor = conn.cursor()
cursor.execute('create database Police')


1

In [11]:
print(Police_logs.columns)   #id stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp

Index(['stop_date', 'stop_time', 'country_name', 'driver_gender',
       'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw',
       'violation', 'search_conducted', 'search_type', 'stop_outcome',
       'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number'],
      dtype='object')


In [10]:
Police_logs = pd.read_csv(
    r"C:/Users/danie/Downloads/project/guvi/pratice/traffic_stops - traffic_stops_with_vehicle_number.csv",
    low_memory=False
)
Police_logs.dropna(axis=1,how='all',inplace=True)
Police_logs.fillna({'driver_age':Police_logs['driver_age'].median(),
                    'search_type':'None',
                    'stop_duration':'Unknown',
                    'violation':'Unknown',
                    'stop_outcome':'Unknown'}, inplace= True)

con = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="634155"
)
cursor = con.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Police")
cursor.execute("USE Police")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Police_log (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age_raw INT,
    driver_age INT,
    driver_race VARCHAR(50),
    violation_raw VARCHAR(255),
    violation VARCHAR(255),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(20),
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

for _, row in Police_logs.iterrows():
    sql = """
    INSERT INTO Police_log 
    (stop_date, stop_time, country_name, driver_gender, driver_age_raw, driver_age, driver_race,
     violation_raw, violation, search_conducted, search_type, stop_outcome, is_arrested,
     stop_duration, drugs_related_stop, vehicle_number)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    values = (
        row['stop_date'], row['stop_time'], row['country_name'], row['driver_gender'],
        int(row['driver_age_raw']) if not pd.isna(row['driver_age_raw']) else None,
        int(row['driver_age']) if not pd.isna(row['driver_age']) else None,
        row['driver_race'], row['violation_raw'], row['violation'],
        bool(row['search_conducted']), row['search_type'], row['stop_outcome'],
        bool(row['is_arrested']), row['stop_duration'], bool(row['drugs_related_stop']),
        row['vehicle_number']
    )
    cursor.execute(sql, values)

con.commit()
cursor.close()
con.close()
print("✅ Data inserted successfully into Police_log")

✅ Data inserted successfully into Police_log


In [13]:
Police_logs

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,0-15 Min,True,KA24UZ8488
